In [1]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
from numpy.lib.arraypad import pad
from python_speech_features import mfcc

In [2]:
audio_path = 'G:\Voice Dataset'
audio_paths = [os.path.join(audio_path, f) for f in os.listdir(audio_path)]

In [3]:
def extract_features(audio_path):
    signal, rate = librosa.load(audio_path, sr=16000)
    mfcc_feat = mfcc(signal, rate, numcep=13, nfilt=26, nfft=512)
    return mfcc_feat

In [4]:
def get_labels(audio_path):
    labels = []
    for file in os.listdir(audio_path):
        if file.endswith('.wav'):
            label = file[0]  # Extract the first character of the file name as the label
            if label == 'u':  # If 'u' is present, skip this label
                continue
            if label == 'f':  # If the label is 'f' then  it is female
                label = 'female'
            elif label == 'm':  # If 'm'  it is male
                label = 'male'
            labels.append(label)
    return np.array(labels)

In [5]:
# Extract MFCC features from the audio files
features = [extract_features(audio_path) for audio_path in audio_paths]

In [6]:
# Determine the maximum length of the features
max_len = max([len(f) for f in features])

In [7]:
# Pad features with zeros to the maximum length
X = np.zeros((len(features), max_len, 13))
for i, f in enumerate(features):
    X[i, :len(f), :] = f

In [8]:
X_flat = X.reshape(X.shape[0], -1)

In [9]:
X_flat.shape

(3842, 12987)

In [10]:
y = get_labels(audio_path)

In [11]:
y.shape

(3842,)

In [12]:
print(y)

['female' 'female' 'female' ... 'male' 'male' 'male']


In [13]:
num_male = np.sum(y == "male")  
num_female = np.sum(y == "female")  
print("Number of males:", num_male)
print("Number of females:", num_female)

Number of males: 1656
Number of females: 2186


In [14]:
le = LabelEncoder()
y = le.fit_transform(y)

In [15]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [16]:
males=np.sum(y == 1)   #1 is male
females=np.sum(y ==0)  #0 is female
print("Number of males:",males)
print("Number of females:",females)

Number of males: 1656
Number of females: 2186


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42,stratify=y)

In [18]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
# Evaluate the performance of the model
acc = accuracy_score(y_test, y_pred)
print('Accuracy of SVM:', acc)

pre = precision_score(y_test, y_pred)
print('Precision of SVM:', pre)

rec = recall_score(y_test, y_pred)
print('Recall of SVM:', rec)

f1 = f1_score(y_test, y_pred)
print('F1-Score of SVM:', f1)


Accuracy of SVM: 0.976592977893368
Precision of SVM: 0.9815384615384616
Recall of SVM: 0.9637462235649547
F1-Score of SVM: 0.9725609756097561


In [19]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       438
           1       0.98      0.96      0.97       331

    accuracy                           0.98       769
   macro avg       0.98      0.98      0.98       769
weighted avg       0.98      0.98      0.98       769

